In [45]:
%matplotlib inline
import xlrd
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os as os
from itertools import product
import scipy.stats as ss
from scipy.stats import norm
from math import e
np.set_printoptions(suppress=True, linewidth=200, edgeitems=100)

In [46]:
import cx_Oracle

conn = cx_Oracle.connect('XXAPPS/f3b7ce2a3b@vdb1.depauw.edu:1521/DPUPL1')
cursor = conn.cursor()

# Data loading from All Lines file and Metronet Phone Users file

In [47]:
lines_df = pd.read_csv('All Lines DePauw.csv')
metronet_df = pd.read_csv('Metronet Phone Users.csv')

# Data cleansing from given files

## Check duplicated numbers

In [48]:
lines_df[lines_df.duplicated(['Directory Number'])]

,Department,Name,Directory Number


In [49]:
metronet_df[metronet_df.duplicated(['Directory Number'])]

,Directory Number,Extension,Department,Type
340,Information Services,NaN,NaN,NaN
578,Information Services,NaN,NaN,NaN


## Drop duplicated and NaN numbers

In [50]:
a = lines_df.drop_duplicates(subset=['Directory Number']).dropna(subset=['Directory Number']).reset_index(drop=True)
b = metronet_df.drop_duplicates(subset=['Directory Number']).dropna(subset=['Directory Number']).reset_index(drop=True)

a.describe(include=['O'])

,Department,Name,Directory Number
count,1263,1263,1263
unique,86,1128,1263
top,POTS,Charter Telethon,(765) 658 5416
freq,393,19,1


In [51]:
b.describe(include=['O'])

,Directory Number,Department,Type
count,1005,993,993
unique,1005,889,85
top,(765) 658 4238,Charter Telethon,POTS
freq,1,19,256


Now we have unique phone numbers in both files.

## Cross-check phone numbers

See if there is any phone number in Metronet file that does not have a match in All Lines file

In [52]:
b['Phone Match Check'] = ['Match' if x in list(a['Directory Number'])
    else 'No Match' for x in b['Directory Number']]

b[b['Phone Match Check'] == 'No Match']

,Directory Number,Extension,Department,Type,Phone Match Check
255,Alumni Engagement,NaN,NaN,NaN,No Match
303,HelpDesk,NaN,NaN,NaN,No Match
335,Information Services,NaN,NaN,NaN,No Match
434,Media Services,NaN,NaN,NaN,No Match
717,Computer Science,NaN,NaN,NaN,No Match


See if there is any phone number in All Lines file that does not have a match in Metronet file

In [53]:
a['Phone Match Check'] = ['Match' if x in list(b['Directory Number'])
    else 'No Match' for x in a['Directory Number']]

a[a['Phone Match Check'] == 'No Match']

,Department,Name,Directory Number,Phone Match Check
11,Academic Affairs Dept,Becky Wallace,(765) 658 6555,No Match
12,Academic Affairs Dept,Ben Hogan,(765) 658 6595,No Match
13,Academic Affairs Dept,Kelley Hall,(765) 658 6606,No Match
43,Admission,School of Music AA,(765) 658 4380,No Match
52,Admission,Jeffrey Adams,(765) 658 6082,No Match
...,...,...,...,...
1219,Religious Studies,Religious Studies,(765) 658 6386,No Match
1238,Sociology and Anthropology,Matthew Oware,(765) 658 6699,No Match
1243,Student Accessibility Services,Student Accessibility,(765) 658 6267,No Match
1244,Student Accessibility Services,Meggan Johnston,(765) 658 6551,No Match


So for the most part, every number in Metronet file has a corresponding number in All Lines file, but not vice versa. There could be a need to use the Metronet file to check whether a number is POTS/VOIP, but in fact, the All Lines file already has POTS as a value in the department field.

Therefore, we can use the All Lines file as the primary source instead of combining two files.

## Create phone type field

In [54]:
a['Phone Type'] = ['POTS' if x == 'POTS' else 'VOIP' for x in a['Department']]

c = a.drop(['Phone Match Check'],axis='columns')

c['Directory Number'] = c['Directory Number'].str.replace('(','',
                        regex=True).str.replace(')','',
                        regex=True).str.replace(' ','',regex=True)

c['Extension'] = [x[-4:] for x in c['Directory Number']]

c['First Name'] = [x.split( )[0] for x in c['Name']]
c['Last Name'] = [x.split( )[-1] for x in c['Name']]
c['Adhoc Name'] = c['First Name'].str[:2]+' '+c['Last Name']

c = c[['Directory Number','Extension','Name','First Name','Last Name','Adhoc Name','Department','Phone Type']]
c

,Directory Number,Extension,Name,First Name,Last Name,Adhoc Name,Department,Phone Type
0,7656584070,4070,Tamara Stasik,Tamara,Stasik,Ta Stasik,Academic Affairs Dept,VOIP
1,7656584091,4091,Corinne Wagner,Corinne,Wagner,Co Wagner,Academic Affairs Dept,VOIP
2,7656584159,4159,Valerie O'Hair,Valerie,O'Hair,Va O'Hair,Academic Affairs Dept,VOIP
3,7656584303,4303,Jeane Pope Lab,Jeane,Lab,Je Lab,Academic Affairs Dept,VOIP
4,7656584358,4358,Bridget Gourley,Bridget,Gourley,Br Gourley,Academic Affairs Dept,VOIP
...,...,...,...,...,...,...,...,...
1258,7656581041,1041,Christina Holmes,Christina,Holmes,Ch Holmes,Women's Studies,VOIP
1259,7656584759,4759,Leigh-Anne Goins,Leigh-Anne,Goins,Le Goins,Women's Studies,VOIP
1260,7656586751,6751,Women's Studies,Women's,Studies,Wo Studies,Women's Studies,VOIP
1261,7656581088,1088,Womens Center,Womens,Center,Wo Center,Womens Center,VOIP


## Fix naming issue

Names have to be consistent with first name and last name in ADP

In [55]:
# Create "adhoc name" in merged dataframe-created from given files (done already) and dataframe employee_name, 
# which is just last name and the first two letters of first name

# Create a column names "Correct Name"
# If directory name matches legal name in ADP, column returns legal name
# If directory name matches nickname in ADP, column returns legal name that corresponds to the nickname
# If directory adhoc name matches adhoc name in ADP, column returns legal name that corresponds to the adhoc name
# Else, column returns "No Match"
# Modify the original "Name" column based on the values in "Correct Name" column

# This algorithm will not resolve names with double (hyphenated) last names, unexpected names,
# names of dependent, and names with special character in directory
# Create a function and a (manual) dictionary to replace some "No Match" values by the correct legal names

# The final dataframe can be considered the Master File to be used

query = """
select first_name, last_name, person_type, known_as, first_name||' '||last_name legal_name, 
case when known_as is not null then known_as||' '||last_name else null end as nickname,
substr(first_name,1,2)||' '||last_name adhoc_name

from xxadp_people 

where sysdate between effective_start_date and effective_end_date
and (person_type like '%mployee%' or person_type like '%orker%' or person_type like '%orking%' 
or person_type like '%meritus%' or person_type like '%eceased%' or person_type like '%etire%')
"""

cursor.execute(query)
records = cursor.fetchall()

employee_name = pd.DataFrame(records)
employee_name.rename(columns={0: "First Name", 1: "Last Name", 2: "Person Type", 3: "Known As",
                              4: "Legal Name", 5: "Nickname", 6: "Adhoc Name"}, inplace=True)

c['Correct Name'] = [x if x in list(employee_name['Legal Name']) else
    employee_name.loc[employee_name['Nickname'] == x, 'Legal Name'].item() if x in list(employee_name['Nickname']) else
    employee_name.loc[employee_name['Adhoc Name'] == y, 'Legal Name'].iloc[0] if y in list(employee_name['Adhoc Name']) else 
    'No Match' for (x,y) in zip(c['Name'],c['Adhoc Name'])]

c['Name'] = [x if y=='No Match' else y for (x,y) in zip(c['Name'],c['Correct Name'])]

def replace_all(column, dictionary):
    for i, j in dictionary.items():
        column = column.str.replace(i, j, regex=True)
    return column

dictionary = {'Tony Lowe':'Anthony Lowe',
              'Tony Tillman':'Anthony Tillman',
             'C.J. Gomolka':'Carl Gomolka',
              'Bill Smith':'William Smith',
              'Dr. Snelson':'David Snelson',
              'Jamie Stuckey':'Jamie Pell-Stuckey',
              'Kevin Brown':'Kenneth Brown',
             'Pat Babington':'James Babington',
              'B. Charoenphon':'Sutthirut Charoenphon',
              'H Hankinson':'Holbrook Hankinson',
             'S Schillerstrom':'Spencer Schillerstrom',
                'Colleen McCracken':'Colleen McCracken Renick',
              'Kat Gruener':'Valerie Gruener',
              'khadija stewart':'Khadija Stewart',
              'Dr. David Harsha':'David Harsha',
              'Nikki Brueggeman':'Nikki Weber-Brueggeman',
              'Sudha Tiruchengodu Yegyanarayanan':'Sudha Anand',
              'Jo MacPhail':'Josephine MacPhail',
             'Kayla Birt':'Kayla Flegal',
                'Jeannette Johnson-Licon':'Jeannette Johnson',
              'Prakash Deepa':'Deepa Prakash',
                'Mar�a Victoria Luque':'Maria Luque',
              'Samantha Sarich':'Samantha Sarich Nance',
              'S Sarich Nance':'Samantha Sarich Nance',
                  'Soledad Forcadell':'Maria Forcadell',
              'Maureen Langdoc - Gobin':'Maureen Knudsen Langdoc',
              'Kara Stolle/Leonid Sirotkin':'Leonid Sirotkin',
             'Caf�':'Cafe'}

c['Name'] = replace_all(c['Name'],dictionary)

d = c[['Directory Number','Extension','Name','Department','Phone Type']].sort_values(by=['Department'],
                        key=lambda col: col.str.lower()).reset_index(drop=True)
d                                                            

,Directory Number,Extension,Name,Department,Phone Type
0,7656584070,4070,Tamara Stasik,Academic Affairs Dept,VOIP
1,7656584091,4091,Corinne Wagner,Academic Affairs Dept,VOIP
2,7656584159,4159,Valerie O'Hair,Academic Affairs Dept,VOIP
3,7656584303,4303,Jeane Pope Lab,Academic Affairs Dept,VOIP
4,7656584358,4358,Bridget Gourley,Academic Affairs Dept,VOIP
...,...,...,...,...,...
1258,7656584759,4759,Leigh-Anne Goins,Women's Studies,VOIP
1259,7656581041,1041,Christina Holmes,Women's Studies,VOIP
1260,7656586751,6751,Women's Studies,Women's Studies,VOIP
1261,7656581088,1088,Womens Center,Womens Center,VOIP


 DataFrame d can be considered the master file and the single source of truth.

# Data loading from ADP Schema

For comparision purposes, data from ADP must satisfy these conditions:
- having "658" as the area code
- belonging to a current employee or a current contingent worker 

In [56]:
query = """
select p.first_name||' '||p.last_name name, p.person_type employment_type, a.address_line1, a.address_line2, a.address_line3, 
a.telephone_number_1, a.telephone_number_2

from xxadp_people p
join xxadp_addresses a
on a.person_id = p.person_id
and (substr(telephone_number_1,1,7) = '765-658' or substr(telephone_number_2,1,7) = '765-658')
and sysdate between a.date_from and nvl(a.date_to, to_date('31-DEC-4712'))

where sysdate between p.effective_start_date and p.effective_end_date
and (p.person_type = 'Employee' or p.person_type like '%Contingent Worker%')

order by telephone_number_1 asc
"""
cursor.execute(query)
records = cursor.fetchall()

e = pd.DataFrame(records)
e.rename(columns={0: "Name", 1: "Employment Type", 2: "Address Line 1", 3: "Address Line 2",
                  4: "Address Line 3", 5: "Campus Phone 1", 6: "Campus Phone 2"}, inplace=True)

e1 = e[~(e['Campus Phone 1'].isnull())][['Name', 'Employment Type',
                                         'Address Line 1','Address Line 2','Address Line 3','Campus Phone 1']]

e1.rename(columns={'Campus Phone 1':'Campus Phone'},inplace=True)

e2 = e[~(e['Campus Phone 2'].isnull())][['Name','Employment Type',
                                         'Address Line 1','Address Line 2','Address Line 3','Campus Phone 2']]

e2.rename(columns={'Campus Phone 2':'Campus Phone'},inplace=True)

e = pd.concat([e1,e2])

e['Campus Phone'] = e['Campus Phone'].replace('-','',regex=True)
e = e[['Campus Phone','Name','Employment Type', 'Address Line 1', 'Address Line 2', 'Address Line 3']]
e

,Campus Phone,Name,Employment Type,Address Line 1,Address Line 2,Address Line 3
0,7656581021,Eliza Brown,Employee,School of Music,"Green Center for Performing Arts, Room 1107",None
1,7656581035,Carl Gomolka,Employee,Global French Studies,"East College, Room 305",None
2,7656581036,Christopher Wolfe,Employee,Office of University Communications,101 E. Seminary Street,None
3,7656581039,Glen Kuecker,Employee,History,"Harrison Hall, Room 221",7 E Larabee Street
4,7656581041,Christina Holmes,Employee,"Women's, Gender and Sexuality Studies","Asbury Hall, Room 205B",None
...,...,...,...,...,...,...
508,7656586757,Leonid Sirotkin,Employee,School of Music,"Green Center for the Performing Arts, Room 1110",None
509,7656586798,Scotty Stepp,Employee,School of Music,"Green Center for the Performing Arts, Room 0161",600 S. Locust Street
510,7656586798,James Beckel,Employee,School of Music,"Green Center for the Performing Arts, Room 0161",701 S College Avenue
511,7656586799,Curtis Carpenter,Employee,Information Services,"Julian Science and Mathematics Center, Room 166",None


# Data inconsistencies

## Numbers in ADP that are NOT in directory

In [57]:
e['Phone Match Check'] = ['Match' if x in list(d['Directory Number'])
    else 'No Match' for x in e['Campus Phone']]

e_nomatch = e[(e['Phone Match Check'] == 'No Match') & (e['Campus Phone'].str.len() == 10)]

e_nondir = e_nomatch.groupby(['Campus Phone']).agg({'Name':'sum'})
e_nondir

,Name
Campus Phone,
7656584500,Angelique SerranoNicole CollisiHaleigh Chastee...
7656584798,John Feuquay
7656585555,Antoine StewartStephen SmithRobert CarterCamer...
7656586570,Jia-Ling Weng


Check the name(s) associated with these non-directory numbers and replace those if necessary

In [58]:
e_nondir.loc['7656584500'] = 'Housing and Residence Life'
e_nondir.loc['7656585555'] = 'DePauw Police'
e_nondir.rename(columns={'Name_x':'Name'},inplace=True)
e_nondir

,Name
Campus Phone,
7656584500,Housing and Residence Life
7656584798,John Feuquay
7656585555,DePauw Police
7656586570,Jia-Ling Weng


We conclude that very few numbers were not recorded in the Master File.

## Numbers in directory that are NOT in ADP

In [59]:
d['Phone Match Check'] = ['Match' if x in list(e['Campus Phone'])
    else 'No Match' for x in d['Directory Number']]

phone_check = d[d['Phone Match Check'] == 'No Match']

col_check = pd.DataFrame(['Match' if x in list(e['Name']) else 'No Match' for x in phone_check['Name']])

phone_check = phone_check.reset_index(drop=True).join(col_check).rename(columns={0:'Name Match Check'})

name_check = phone_check[phone_check['Name Match Check'] == 'Match'][['Directory Number',
            'Extension','Name','Department','Phone Type']]

d_nonadp = name_check.sort_values(by=['Department'],key=lambda col: col.str.lower()).reset_index(drop=True)
d.drop(columns={'Phone Match Check'},inplace=True)

d_nonadp

,Directory Number,Extension,Name,Department,Phone Type
0,7656584070,4070,Tamara Stasik,Academic Affairs Dept,VOIP
1,7656584358,4358,Bridget Gourley,Academic Affairs Dept,VOIP
2,7656584601,4601,David Berque,Academic Affairs Dept,VOIP
3,7656584112,4112,Benjamin Hogan,Academic Services,VOIP
4,7656584034,4034,Dennis Dechant,Art and Art History,VOIP
...,...,...,...,...,...
58,7656584143,4143,Kelley Hall,Registrar's Office,VOIP
59,7656584835,4835,Raymond Miller,Service Center,VOIP
60,7656586551,6551,Meggan Johnston,Student Accessibility Services,VOIP
61,7656584804,4804,Haleigh Chasteen,Student Life,VOIP


These people's ADP information needs to be updated. They either have a new phone number, or have two phone numbers and the second one hasn't been recorded.

## Numbers in ADP and in directory but under different names

In [60]:
combined_df = e[['Name','Campus Phone']].merge(d,how='left',left_on='Campus Phone',
            right_on='Directory Number')

combined_df1 = combined_df[~(combined_df['Name_y'].isnull()) & (combined_df['Name_x']!=combined_df['Name_y'])]

combined_df2 = combined_df1[~combined_df1['Name_y'].isin([
    'Psychology',
    'Center for Spiritual Life',
    'Admissions Front Desk',
    'Presidents Office',
    'Facilities AA',
    'Grounds Department',
    'Plumbers',
    'Counseling Services TOD',
    'HelpDesk TOD',
    'Prevo Library Service Desk',
    'Student Post Office phone',
    'School of Music AA',
    'Libraries',
    'Facilities Annex',
    'Dock',
    'Admissions New Hire',
    'English',
    'Biology',
    'Media Services',
    'Political Science',
    'Econ and Management',
    'General Athletics',
    'Lily Equipment Room',
    'Art Department',
    'Peeler Front Desk',
    'Student Accessibility',
    'Economics and Management'
])]

combined_df3 = combined_df2.rename(columns={'Name_x':'Name from ADP','Name_y':'Name from Master File'})

combined_df4 = combined_df3[['Campus Phone','Extension','Name from ADP','Name from Master File','Department']]

dif_df = combined_df4.sort_values(by=['Department'],key=lambda col: col.str.lower()).reset_index(drop=True)
dif_df

,Campus Phone,Extension,Name from ADP,Name from Master File,Department
0,7656584359,4359,David Berque,Brenda Rogers,Academic Affairs Dept
1,7656586606,6606,Matthew Meier,Kelley Hall,Academic Affairs Dept
2,7656586606,6606,Susan Wilson,Kelley Hall,Academic Affairs Dept
3,7656584336,4336,John Berry,Misti Scott,Art and Art History
4,7656584345,4345,Dennis Dechant,Anne Harris,Art and Art History
5,7656584012,4012,Nipun Chopra,Melynda Link,Athletics
6,7656584921,4921,Vincent Lazar,Clint Wallman,Athletics
7,7656584940,4940,Dennis Harrington,Russell Loyd,Athletics
8,7656586280,6280,Anne Bourne,Cameron Tucker,Athletics
9,7656584166,4166,Monica Jorgensen,Linan Peng,Economics and Management


# Directory Clean-Up

## Add missing numbers, location and address

In [61]:
# Add numbers in ADP that are not in directory
data_to_add = pd.DataFrame({'Directory Number':['7656584500','7656584798','7656585555','7656586570'],
        'Extension':['4500', '4798', '5555', '6570'],
        'Name':['Housing', 'John Feuquay', 'Public Safety', 'Jia-Ling Weng'],
        'Department':['Housing and Residence Living','Media Services','Public Safety','Modern Languages'],
        'Phone Type':['VOIP','VOIP','VOIP','VOIP']})

d_added = d.append(data_to_add, ignore_index = True).sort_values(
    by=['Department'],key=lambda col: col.str.lower()).reset_index(drop=True)

# Create person type column (Faculty, Staff, Organization Number, POTS Number)
query = """
select pv.name, 
case when title like '%rofessor%' or employment_type like '%meritus%'then 'Faculty' 
else 'Staff' end as person_type

from
(select p.person_id, min(p.first_name||' '||p.last_name) name, min(p.person_type) employment_type, 
listagg(ass.ass_attribute1,', ') title
    
from xxadp_people p

left join xxadp_assignments ass
on p.person_id = ass.person_id
and sysdate between ass.effective_start_date and ass.effective_end_date

where sysdate between p.effective_start_date and p.effective_end_date    
and (person_type like '%mployee%' or person_type like '%orker%' or person_type like '%orking%' 
or person_type like '%meritus%' or person_type like '%eceased%' or person_type like '%etire%')

group by p.person_id) pv
"""

cursor.execute(query)
records = cursor.fetchall()

type_df = pd.DataFrame(records)
type_df.rename(columns={0: "Name", 1: "Person Type"}, inplace=True)


d_added['Person Type'] = [type_df.loc[type_df['Name'] == x, 'Person Type'].iloc[0] if x in list(type_df['Name']) 
                            else 'POTS Line' if d_added.loc[d_added['Name'] == x, 'Phone Type'].iloc[0] == 'POTS'
                              else 'Non-DPU or Org. Number' for x in d_added['Name']]

# Fix some department naming
d_added.loc[d_added['Department'] == 'Athletics / Blackstock', 'Department'] ='Athletics'
d_added.loc[d_added['Department'] == 'Conflict Studies', 'Department'] ='Peace and Conflict Studies'
d_added.loc[d_added['Department'] == 'DePauw Police Department', 'Department'] ='DePauw Police'
d_added.loc[d_added['Department'] == 'Information Services / Returned June 2021', 'Department'] ='Information Services'
d_added.loc[d_added['Department'] == 'Music', 'Department'] ='School of Music'
d_added.loc[d_added['Department'] == 'Print Shop', 'Department'] ='Printing Services'
d_added.loc[d_added['Department'] == 'Psychology', 'Department'] ='Psychology and Neuroscience'
d_added.loc[d_added['Department'] == 'Public Safety', 'Department'] ='DePauw Police'
d_added.loc[d_added['Department'] == 'Student Life', 'Department'] ='Student Affairs'
d_added.loc[d_added['Department'] == 'Student Life Division', 'Department'] ='Student Affairs'
d_added.loc[d_added['Department'] == 'Women\'s Studies', 'Department'] ='Women\'s, Gender and Sexuality Studies'
d_added.loc[d_added['Department'] == 'American Academic Leadership', 'Department'] ='Art and Art History'
d_added.loc[d_added['Department'] == 'Conference and Campus Events', 'Department'] ='Facilities Management'
d_added.loc[d_added['Department'] == 'Development and Alumni Engagement', 'Department'] ='Development'
d_added.loc[d_added['Department'] == 'Development Services', 'Department'] ='Development'
d_added.loc[d_added['Department'] == 'Finance', 'Department'] ='Finance and Administration'
d_added.loc[d_added['Department'] == 'Instructional & Learning Svcs', 'Department'] ='Information Services'
d_added.loc[d_added['Department'] == 'Mail Center', 'Department'] ='Facilities Management'
d_added.loc[d_added['Department'] == 'Office of Spiritual Life', 'Department'] ='Center for Spiritual Life'
d_added.loc[d_added['Department'] == 'Performing Arts', 'Department'] ='School of Music'
d_added.loc[d_added['Department'] == 'Service Center', 'Department'] ='Facilities Management'

# Get location and address data from database
query = """
select name, max(location_code), max(address_line_1) 
from xxadp_organizations
where location_code is not null
group by name
order by name
"""

cursor.execute(query)
records = cursor.fetchall()

org = pd.DataFrame(records)
org.rename(columns={0: "Department", 1: "Location", 2: "Address"}, inplace=True)

# Join directory data with department location data
d_merge = d_added.merge(org,how='left',left_on='Department',right_on='Department')

# Manually fix some missing locations
d_merge.loc[d_merge['Department'] == 'bonappetit','Location'] = 'Hoover Hall'
d_merge.loc[d_merge['Department'] == 'bonappetit','Address'] = '101 E Hanna St'

d_merge.loc[d_merge['Department'] == 'Gobin','Location'] = 'Gobin Church'
d_merge.loc[d_merge['Department'] == 'Gobin','Address'] = '305 Simpson St'

d_merge.loc[d_merge['Department'] == 'HelpDesk','Location'] = 'Julian'
d_merge.loc[d_merge['Department'] == 'HelpDesk','Address'] = '602 S College Ave'

d_merge.loc[d_merge['Department'] == 'Housing and Residence Living','Location'] = 'Anderson Street Hall'
d_merge.loc[d_merge['Department'] == 'Housing and Residence Living','Address'] = '413 Anderson St'

d_merge.loc[d_merge['Department'] == 'Media Services','Location'] = 'Julian'
d_merge.loc[d_merge['Department'] == 'Media Services','Address'] = '602 S College Ave'

d_merge = d_merge[['Directory Number','Extension','Name','Person Type','Department','Phone Type','Location','Address']]
d_merge

,Directory Number,Extension,Name,Person Type,Department,Phone Type,Location,Address
0,7656584070,4070,Tamara Stasik,Faculty,Academic Affairs Dept,VOIP,Harrison Hall,7 E Larabee St
1,7656584091,4091,Corinne Wagner,Staff,Academic Affairs Dept,VOIP,Harrison Hall,7 E Larabee St
2,7656584159,4159,Valerie O'Hair,Staff,Academic Affairs Dept,VOIP,Harrison Hall,7 E Larabee St
3,7656584303,4303,Jeane Pope Lab,Not Staff or Faculty,Academic Affairs Dept,VOIP,Harrison Hall,7 E Larabee St
4,7656584358,4358,Bridget Gourley,Faculty,Academic Affairs Dept,VOIP,Harrison Hall,7 E Larabee St
...,...,...,...,...,...,...,...,...
1262,7656584759,4759,Leigh-Anne Goins,Faculty,"Women's, Gender and Sexuality Studies",VOIP,Asbury Hall,100 East Seminary Street
1263,7656581041,1041,Christina Holmes,Faculty,"Women's, Gender and Sexuality Studies",VOIP,Asbury Hall,100 East Seminary Street
1264,7656586751,6751,Women's Studies,Not Staff or Faculty,"Women's, Gender and Sexuality Studies",VOIP,Asbury Hall,100 East Seminary Street
1265,7656581088,1088,Womens Center,Not Staff or Faculty,Womens Center,VOIP,The Womens Center,306 E Hanna Street


## Add student phone numbers

Only students entered DePauw in/after Fall 2017-2018 are considered

In [67]:
query = """
select replace(replace(replace(replace(d.comm_value,'-',''),' ',''),'+',''),'.','') directory_number, null extension,
p.first_name||' '||p.last_name name, 'Student' person_type, 'Student' department, 'VOIP' phone_type,

from xxcs_personmaster_v p
join xxcis_digicomm d
on p.person_id = d.person_id
and sysdate between d.date_from and d.date_to
and d.comm_type_id = 2141

where p.cs_entrance_semester_code >= 2017201820
"""

cursor.execute(query)
records = cursor.fetchall()
records

[('17657202837',
  None,
  'James Tate',
  'Student',
  'Student',
  'VOIP',
  'Student Location',
  'Student Address'),
 ('3179027522',
  None,
  'Annika Whitlock',
  'Student',
  'Student',
  'VOIP',
  'Student Location',
  'Student Address'),
 ('7657193328',
  None,
  'Ben Gellman',
  'Student',
  'Student',
  'VOIP',
  'Student Location',
  'Student Address'),
 ('8475086536',
  None,
  'Nicole Nale',
  'Student',
  'Student',
  'VOIP',
  'Student Location',
  'Student Address'),
 ('3176540789',
  None,
  'SyRia Hudson',
  'Student',
  'Student',
  'VOIP',
  'Student Location',
  'Student Address'),
 ('3039496502',
  None,
  'Samuel Yeager',
  'Student',
  'Student',
  'VOIP',
  'Student Location',
  'Student Address'),
 ('923005555769',
  None,
  'Khadija Abid',
  'Student',
  'Student',
  'VOIP',
  'Student Location',
  'Student Address'),
 ('8122437784',
  None,
  'Ryan Scott',
  'Student',
  'Student',
  'VOIP',
  'Student Location',
  'Student Address'),
 ('7737048517',
  None,

In [62]:
query = """
select replace(replace(replace(replace(d.comm_value,'-',''),' ',''),'+',''),'.','') directory_number, null extension,
p.first_name||' '||p.last_name name, 'Student' person_type, 'Student' department, 'VOIP' phone_type,
'Student Location' location, 'Student Address' address

from xxcs_personmaster_v p
join xxcis_digicomm d
on p.person_id = d.person_id
and sysdate between d.date_from and d.date_to
and d.comm_type_id = 2141

where p.cs_entrance_semester_code >= 2017201820
"""

cursor.execute(query)
records = cursor.fetchall()

student_df = pd.DataFrame(records)
student_df.rename(columns={0: "Directory Number", 1: "Extension", 2: "Name", 3: "Person Type",
                          4:  "Department", 5: "Phone Type", 6: "Location", 7: "Address"}, inplace=True)
student_df

,Directory Number,Extension,Name,Person Type,Department,Phone Type,Location,Address
0,7657214586,None,Sterling Silver - Test,Student,Student,VOIP,Student Location,Student Address
1,5744409696,None,Mason Lee,Student,Student,VOIP,Student Location,Student Address
2,3146507860,None,Gabriela Carretero,Student,Student,VOIP,Student Location,Student Address
3,8473467975,None,Andrew Douglas,Student,Student,VOIP,Student Location,Student Address
4,6165405835,None,Thomas Alkema,Student,Student,VOIP,Student Location,Student Address
...,...,...,...,...,...,...,...,...
2691,7734410420,None,Tamariya Handy,Student,Student,VOIP,Student Location,Student Address
2692,9196274369,None,Bisma Ranjah,Student,Student,VOIP,Student Location,Student Address
2693,3123420457,None,Aolani Cano,Student,Student,VOIP,Student Location,Student Address
2694,3177355944,None,Conner Nicoson,Student,Student,VOIP,Student Location,Student Address


In [63]:
directory_df = pd.concat([d_merge,student_df])
directory_df

,Directory Number,Extension,Name,Person Type,Department,Phone Type,Location,Address
0,7656584070,4070,Tamara Stasik,Faculty,Academic Affairs Dept,VOIP,Harrison Hall,7 E Larabee St
1,7656584091,4091,Corinne Wagner,Staff,Academic Affairs Dept,VOIP,Harrison Hall,7 E Larabee St
2,7656584159,4159,Valerie O'Hair,Staff,Academic Affairs Dept,VOIP,Harrison Hall,7 E Larabee St
3,7656584303,4303,Jeane Pope Lab,Not Staff or Faculty,Academic Affairs Dept,VOIP,Harrison Hall,7 E Larabee St
4,7656584358,4358,Bridget Gourley,Faculty,Academic Affairs Dept,VOIP,Harrison Hall,7 E Larabee St
...,...,...,...,...,...,...,...,...
2691,7734410420,None,Tamariya Handy,Student,Student,VOIP,Student Location,Student Address
2692,9196274369,None,Bisma Ranjah,Student,Student,VOIP,Student Location,Student Address
2693,3123420457,None,Aolani Cano,Student,Student,VOIP,Student Location,Student Address
2694,3177355944,None,Conner Nicoson,Student,Student,VOIP,Student Location,Student Address


In [64]:
directory_df.to_csv("Directory Master.csv")